# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [ ]:
import requests
import time
import re
from functools import wraps

from random import randint

In [ ]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [ ]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
      start = time.perf_counter()
      result = func(*args, **kwargs)
      end = time.perf_counter()
      print(f'{func.__name__} потребовалось {end - start:.6f} секунд')
      return result
    return wrapper

In [ ]:
@benchmark
def get_webpage():
    import requests
    webpage = requests.get('https://stepik.org')

get_webpage()

get_webpage потребовалось 0.891634 секунд


## Задание 2

In [ ]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
      print(f"Функция {func.__name__} была вызвана с аргументами: args={args}, kwargs={kwargs}")
      return func(*args, **kwargs)
    return wrapper

In [ ]:
@logging
def countdown(n, step):
    while n>0:
      n -= step
countdown(10**5, 2)

Функция countdown была вызвана с аргументами: args=(100000, 2), kwargs={}


## Задание 3

In [ ]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    def wrapper(*args, **kwargs):
      wrapper.count += 1
      print(f"Функция {func.__name__} вызвана {wrapper.count} раз(а)")
      return func(*args, **kwargs)
    wrapper.count = 0
    return wrapper

In [ ]:
@counter
def f():
    print("Hello")


f()
f()

print(f.count)

Функция f вызвана 1 раз(а)
Hello
Функция f вызвана 2 раз(а)
Hello
2


## Задание 4

In [ ]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  def fmemo(*args):
    if args not in cache:
        cache[args] = func(*args)
    return cache[args]

  fmemo.cache = cache
  return fmemo

## Тестирование

In [ ]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция wrapper вызвана 1 раз(а)
Функция word_count была вызвана с аргументами: args=('whole',), kwargs={}
word_count потребовалось 0.620679 секунд
Cлово whole встречается 176 раз


In [ ]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [ ]:
%%time
# измеряем время выполнения
print (fib(10))

55
CPU times: user 990 µs, sys: 0 ns, total: 990 µs
Wall time: 999 µs


In [ ]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [ ]:
%%time
# измеряем время выполнения
print (fib(10))

55
CPU times: user 685 µs, sys: 0 ns, total: 685 µs
Wall time: 567 µs
